In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, QuatE, RotatE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive_io.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True
                     )


In [5]:
model_name = 'rotatE_transductive_io'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200

In [8]:
model = RotatE(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

Number of parameters: 6580800
Space occupied: 26323200 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory rotatE_transductive_io created successfully!


In [10]:
learning_rate = 5e-2 #1e-3, and 1e-4 stops after 20 iterations and actually does not reduce the error too much
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.5015852451324463
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.00023313885480321855
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00014613883905568198
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.00018963884692945024
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00023313862038776278
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.00014613871462643147
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0001896386529551819
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.00023313835359902433
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00014613856924195615
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00018963846142049025
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validat

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 2.8994946479797363
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.00022810636794825108
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.00010941926455321396
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.0001687628162507325
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.00022810602968093008
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.0001094191538868472
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.0001687625772319734
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.00022810568377858785
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.00010941904013143599
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.00016876236195501195
Metric: validation.head.optimistic.hits_at_1 = 0.0
Metric: validation.tail.optimistic.hits_at_1 = 0.0
Metric: validation.both.optimistic.hits_at_1 = 0.0
Metric: validatio

...

Step: 151
Metric: loss = 0.054433420300483704
Step: 151
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.24526749975777637
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.3975154120598432
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3213914559088098
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.24526748061180112
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.3975153863430023
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.3213914632797241
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.24526749880412022
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.3975154120598432
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.32139145543198167
Metric: validation.head.optimistic.hits_at_1 = 0.144316854807318
Metric: validation.tail.optimistic.hits_at_1 = 0.27724795640326977
Metric: validation.both.optimistic.hits_at_1 = 0.210782405605

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 152
Metric: loss = 0.05531097948551178
Step: 152
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.24543820556553564
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.3973148273796077
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.32137651647257165
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.24543820321559906
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.39731481671333313
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.3213765323162079
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.24543820503800198
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.39731482719483463
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.3213765161164183
Metric: validation.head.optimistic.hits_at_1 = 0.144024912417283
Metric: validation.tail.optimistic.hits_at_1 = 0.2770533281432464
Metric: validation.both.optimistic.hits_at_1 = 0.210539120280

Training batches on cuda:0:   0%|          | 0/1 [00:00<?, ?batch/s]

Step: 153
Metric: loss = 0.05468849465250969
Step: 153
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.245627438659942
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.39712367934553094
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3213755590027365
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.24562744796276093
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.39712339639663696
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.32137542963027954
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.24562743746605004
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.39712314413466576
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.32137529080035787
Metric: validation.head.optimistic.hits_at_1 = 0.144024912417283
Metric: validation.tail.optimistic.hits_at_1 = 0.276274815103153
Metric: validation.both.optimistic.hits_at_1 = 0.2101498637602

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
            0
0    3.501585
1    2.899495
2    2.301180
3    1.721717
4    1.209039
..        ...
148  0.054282
149  0.055865
150  0.054433
151  0.055311
152  0.054688

[153 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/41.5k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.731696   0.731695     0.731696
hits_at_1                     0.660830   0.660830     0.660830
hits_at_3                     0.769173   0.769173     0.769173
hits_at_5                     0.813759   0.813759     0.813759
hits_at_10                    0.867930   0.867930     0.867930
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.780357   0.780357     0.780357
hits_at_1                     0.721264   0.721264     0.721264
hits_at_3                     0.814987   0.814987     0.814987
hits_at_5                     0.849239   0.849239     0.849239
hits_at_10                    0.890283   0.890283     0.890283
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.756026   0.756026     0.756026
hits_at_1                     0.691047   0.691047     0.691047
hits_at_3                     0.792080  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/10.3k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.241538   0.241529     0.241522
hits_at_1                     0.143052   0.143052     0.143052
hits_at_3                     0.264694   0.264694     0.264694
hits_at_5                     0.335539   0.335539     0.335539
hits_at_10                    0.444531   0.444531     0.444531
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.395936   0.395936     0.395936
hits_at_1                     0.277832   0.277832     0.277832
hits_at_3                     0.458641   0.458641     0.458641
hits_at_5                     0.540969   0.540969     0.540969
hits_at_10                    0.631666   0.631666     0.631666
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.318737   0.318732     0.318729
hits_at_1                     0.210442   0.210442     0.210442
hits_at_3                     0.361668  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/13.1k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.374865   0.374865     0.374865
hits_at_1                     0.273360   0.273360     0.273360
hits_at_3                     0.409849   0.409849     0.409849
hits_at_5                     0.481725   0.481725     0.481725
hits_at_10                    0.585640   0.585640     0.585640
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.475520   0.475520     0.475520
hits_at_1                     0.374828   0.374828     0.374828
hits_at_3                     0.538920   0.538920     0.538920
hits_at_5                     0.589693   0.589693     0.589693
hits_at_10                    0.652317   0.652317     0.652317
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.425193   0.425193     0.425193
hits_at_1                     0.324094   0.324094     0.324094
hits_at_3                     0.474384  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                   embedding_dim   
17                   learning_rate   
18                       optimizer   
19                      num_epochs   
20                        patience   

                                                value  
0                                              cuda:0  
1                                            263232

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)